In [1]:
import pandas as pd
import numpy as np
import findspark
findspark.init('C:\spark\spark-3.1.2-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [2]:
student = spark.read.option("inferschema","true").option("header","true").csv("students.csv")
program = spark.read.option("inferschema","true").option("header","true").csv("program.csv")
student.show(5)
program.show(5)

+---+-----+---------+
| ID| Name|Grad_Prog|
+---+-----+---------+
|  1|  Tom|        1|
|  2| Bill|        2|
|  3| Bond|        3|
|  4|James|        4|
|  5|Steve|        5|
+---+-----+---------+
only showing top 5 rows

+---------+-------+--------+
|Grad_Prog| Degree|  School|
+---------+-------+--------+
|        1|Masters|     MIT|
|        2|Masters|     MIT|
|        3|Masters|Stanford|
|        4|    Phd| Harvard|
|        5|    Phd|Stanford|
+---------+-------+--------+
only showing top 5 rows



In [7]:
student.createOrReplaceTempView("STUDENT")
program.createOrReplaceTempView("PROGRAM")

# Join Types

In [40]:
#join types include: 
#'inner', 'outer', 'full', 'fullouter', 'full_outer', 
#'leftouter', 'left', 'left_outer', 'rightouter', 
#'right', 'right_outer', 
#'leftsemi', 'left_semi', 'semi', 
#'leftanti', 'left_anti', 'anti', 
#'cross'.

In [15]:
#Natural Join
#Matches Column with same name
#No condition is provided, only works when there is a column in both tables with same name
#Here we will match grad_prog from both tables

spark.sql('SELECT * FROM STUDENT NATURAL JOIN PROGRAM').show()

+---------+---+-----+-------+--------+
|Grad_Prog| ID| Name| Degree|  School|
+---------+---+-----+-------+--------+
|        1|  1|  Tom|Masters|     MIT|
|        2|  2| Bill|Masters|     MIT|
|        3|  3| Bond|Masters|Stanford|
|        4|  4|James|    Phd| Harvard|
|        5|  5|Steve|    Phd|Stanford|
|        6|  6|Chris|    Phd|Stanford|
|        7|  7|Peter|Diploma|Stanford|
|        8|  8| Paul|Diploma|     MIT|
|        9|  9|  Rob|    Phd|     MIT|
|       10| 10|  Max|Masters| Harvard|
+---------+---+-----+-------+--------+



In [21]:
#Inner Join
#Join with a condition

student.join(program, student.ID == program.Grad_Prog,'inner').show()
#student.join(program, student.ID == program.Grad_Prog).show()
#program.join(student, program.Grad_Prog == student.ID,'inner').show()

spark.sql('SELECT * FROM STUDENT INNER JOIN PROGRAM ON STUDENT.ID == PROGRAM.Grad_Prog').show()
#spark.sql('SELECT * FROM STUDENT JOIN PROGRAM ON STUDENT.ID == PROGRAM.Grad_Prog').show()

+---+-----+---------+---------+-------+--------+
| ID| Name|Grad_Prog|Grad_Prog| Degree|  School|
+---+-----+---------+---------+-------+--------+
|  1|  Tom|        1|        1|Masters|     MIT|
|  2| Bill|        2|        2|Masters|     MIT|
|  3| Bond|        3|        3|Masters|Stanford|
|  4|James|        4|        4|    Phd| Harvard|
|  5|Steve|        5|        5|    Phd|Stanford|
|  6|Chris|        6|        6|    Phd|Stanford|
|  7|Peter|        7|        7|Diploma|Stanford|
|  8| Paul|        8|        8|Diploma|     MIT|
|  9|  Rob|        9|        9|    Phd|     MIT|
| 10|  Max|       10|       10|Masters| Harvard|
+---+-----+---------+---------+-------+--------+

+---+-----+---------+---------+-------+--------+
| ID| Name|Grad_Prog|Grad_Prog| Degree|  School|
+---+-----+---------+---------+-------+--------+
|  1|  Tom|        1|        1|Masters|     MIT|
|  2| Bill|        2|        2|Masters|     MIT|
|  3| Bond|        3|        3|Masters|Stanford|
|  4|James|        

In [29]:
#Right Outer Join
#Outputs whole right table ,fills null if left one does not have values
student.join(program, student.ID == program.Grad_Prog,'right_outer').show()
#student.join(program, student.ID == program.Grad_Prog,'right').show()

#spark.sql('SELECT * FROM STUDENT RIGHT OUTER JOIN PROGRAM ON STUDENT.ID == PROGRAM.Grad_Prog').show()

#Left Outer Join
#Outputs whole left
program.join(student, program.Grad_Prog == student.ID,'left_outer').show()
#program.join(student, program.Grad_Prog == student.ID,'left').show()

+----+-----+---------+---------+---------+--------+
|  ID| Name|Grad_Prog|Grad_Prog|   Degree|  School|
+----+-----+---------+---------+---------+--------+
|   1|  Tom|        1|        1|  Masters|     MIT|
|   2| Bill|        2|        2|  Masters|     MIT|
|   3| Bond|        3|        3|  Masters|Stanford|
|   4|James|        4|        4|      Phd| Harvard|
|   5|Steve|        5|        5|      Phd|Stanford|
|   6|Chris|        6|        6|      Phd|Stanford|
|   7|Peter|        7|        7|  Diploma|Stanford|
|   8| Paul|        8|        8|  Diploma|     MIT|
|   9|  Rob|        9|        9|      Phd|     MIT|
|  10|  Max|       10|       10|  Masters| Harvard|
|null| null|     null|       11|Bachelors| Harvard|
|null| null|     null|       12|      Phd|     MIT|
|null| null|     null|       13|  Diploma|     MIT|
+----+-----+---------+---------+---------+--------+

+---------+---------+--------+----+-----+---------+
|Grad_Prog|   Degree|  School|  ID| Name|Grad_Prog|
+---------+

In [37]:
#Semi Joins
#They are like filters
student.join(program, student.ID == program.Grad_Prog,'left_semi').show()
#student.join(program, student.ID == program.Grad_Prog,'semi').show()

#spark.sql('SELECT * FROM STUDENT LEFT SEMI JOIN PROGRAM ON STUDENT.ID == PROGRAM.Grad_Prog').show()

+---+-----+---------+
| ID| Name|Grad_Prog|
+---+-----+---------+
|  1|  Tom|        1|
|  2| Bill|        2|
|  3| Bond|        3|
|  4|James|        4|
|  5|Steve|        5|
|  6|Chris|        6|
|  7|Peter|        7|
|  8| Paul|        8|
|  9|  Rob|        9|
| 10|  Max|       10|
+---+-----+---------+



In [38]:
#Anti joins
#Opposite of semi
student.join(program, student.ID == program.Grad_Prog,'left_anti').show()

#spark.sql('SELECT * FROM STUDENT LEFT ANTI JOIN PROGRAM ON STUDENT.ID == PROGRAM.Grad_Prog').show()

+---+----+---------+
| ID|Name|Grad_Prog|
+---+----+---------+
+---+----+---------+



# Physical Plan of Each Join

In [44]:
#Natural Join
spark.sql('SELECT * FROM STUDENT NATURAL JOIN PROGRAM').explain()

== Physical Plan ==
*(2) Project [Grad_Prog#18, ID#16, Name#17, Degree#39, School#40]
+- *(2) BroadcastHashJoin [Grad_Prog#18], [Grad_Prog#38], Inner, BuildLeft, false
   :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[2, int, false] as bigint)),false), [id=#2338]
   :  +- *(1) Filter isnotnull(Grad_Prog#18)
   :     +- FileScan csv [ID#16,Name#17,Grad_Prog#18] Batched: false, DataFilters: [isnotnull(Grad_Prog#18)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/students.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Grad_Prog)], ReadSchema: struct<ID:int,Name:string,Grad_Prog:int>
   +- *(2) Filter isnotnull(Grad_Prog#38)
      +- FileScan csv [Grad_Prog#38,Degree#39,School#40] Batched: false, DataFilters: [isnotnull(Grad_Prog#38)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/program.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Grad_Prog)], ReadSchema: struct<Grad_Prog:int,Degree:string,School:string>




In [45]:
#Inner Join
student.join(program, student.ID == program.Grad_Prog,'inner').explain()

== Physical Plan ==
*(2) BroadcastHashJoin [ID#16], [Grad_Prog#38], Inner, BuildLeft, false
:- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#2370]
:  +- *(1) Filter isnotnull(ID#16)
:     +- FileScan csv [ID#16,Name#17,Grad_Prog#18] Batched: false, DataFilters: [isnotnull(ID#16)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/students.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ID)], ReadSchema: struct<ID:int,Name:string,Grad_Prog:int>
+- *(2) Filter isnotnull(Grad_Prog#38)
   +- FileScan csv [Grad_Prog#38,Degree#39,School#40] Batched: false, DataFilters: [isnotnull(Grad_Prog#38)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/program.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Grad_Prog)], ReadSchema: struct<Grad_Prog:int,Degree:string,School:string>




In [48]:
#Right Outer Join
student.join(program, student.ID == program.Grad_Prog,'right_outer').explain()

== Physical Plan ==
*(2) BroadcastHashJoin [ID#16], [Grad_Prog#38], RightOuter, BuildLeft, false
:- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#2454]
:  +- *(1) Filter isnotnull(ID#16)
:     +- FileScan csv [ID#16,Name#17,Grad_Prog#18] Batched: false, DataFilters: [isnotnull(ID#16)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/students.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ID)], ReadSchema: struct<ID:int,Name:string,Grad_Prog:int>
+- FileScan csv [Grad_Prog#38,Degree#39,School#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/program.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Grad_Prog:int,Degree:string,School:string>




In [49]:
#semi joins
student.join(program, student.ID == program.Grad_Prog,'left_semi').explain()

== Physical Plan ==
*(2) BroadcastHashJoin [ID#16], [Grad_Prog#38], LeftSemi, BuildRight, false
:- *(2) Filter isnotnull(ID#16)
:  +- FileScan csv [ID#16,Name#17,Grad_Prog#18] Batched: false, DataFilters: [isnotnull(ID#16)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/students.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ID)], ReadSchema: struct<ID:int,Name:string,Grad_Prog:int>
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#2486]
   +- *(1) Filter isnotnull(Grad_Prog#38)
      +- FileScan csv [Grad_Prog#38] Batched: false, DataFilters: [isnotnull(Grad_Prog#38)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/program.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Grad_Prog)], ReadSchema: struct<Grad_Prog:int>




In [50]:
#anti
student.join(program, student.ID == program.Grad_Prog,'left_anti').explain()

== Physical Plan ==
*(2) BroadcastHashJoin [ID#16], [Grad_Prog#38], LeftAnti, BuildRight, false
:- FileScan csv [ID#16,Name#17,Grad_Prog#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/students.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<ID:int,Name:string,Grad_Prog:int>
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#2512]
   +- *(1) Filter isnotnull(Grad_Prog#38)
      +- FileScan csv [Grad_Prog#38] Batched: false, DataFilters: [isnotnull(Grad_Prog#38)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/program.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Grad_Prog)], ReadSchema: struct<Grad_Prog:int>




# Physical Plan Of Basic Spark/SQL Queries

In [57]:
bus = spark.read.option("inferschema","true").option("header","true").csv("bus.csv")
bus.createTempView('BUS')

In [54]:
program.groupBy('Degree').count().explain()
program.groupBy('Degree').sum().explain()

== Physical Plan ==
*(2) HashAggregate(keys=[Degree#39], functions=[count(1)])
+- Exchange hashpartitioning(Degree#39, 200), ENSURE_REQUIREMENTS, [id=#2550]
   +- *(1) HashAggregate(keys=[Degree#39], functions=[partial_count(1)])
      +- FileScan csv [Degree#39] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/program.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Degree:string>


== Physical Plan ==
*(2) HashAggregate(keys=[Degree#39], functions=[sum(cast(Grad_Prog#38 as bigint))])
+- Exchange hashpartitioning(Degree#39, 200), ENSURE_REQUIREMENTS, [id=#2569]
   +- *(1) HashAggregate(keys=[Degree#39], functions=[partial_sum(cast(Grad_Prog#38 as bigint))])
      +- FileScan csv [Grad_Prog#38,Degree#39] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/program.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Grad_Prog:int,Degree:string>




In [58]:
#How many breakdowns occurred in total in each town
bus.select(col("Town"),col("Busbreakdown_ID")).groupBy(col("Town")).count().explain()

== Physical Plan ==
*(2) HashAggregate(keys=[Town#2418], functions=[count(1)])
+- Exchange hashpartitioning(Town#2418, 200), ENSURE_REQUIREMENTS, [id=#2648]
   +- *(1) HashAggregate(keys=[Town#2418], functions=[partial_count(1)])
      +- FileScan csv [Town#2418] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/bus.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Town:string>




In [60]:
#How many breakdowns occurred in total in each town per year
bus.select(col("School_Year"),col("Town"),col("Busbreakdown_ID")).groupBy("School_Year","Town").count().orderBy("School_Year").explain()

== Physical Plan ==
*(3) Sort [School_Year#2411 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(School_Year#2411 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#2697]
   +- *(2) HashAggregate(keys=[School_Year#2411, Town#2418], functions=[count(1)])
      +- Exchange hashpartitioning(School_Year#2411, Town#2418, 200), ENSURE_REQUIREMENTS, [id=#2693]
         +- *(1) HashAggregate(keys=[School_Year#2411, Town#2418], functions=[partial_count(1)])
            +- FileScan csv [School_Year#2411,Town#2418] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/bus.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<School_Year:int,Town:string>




In [63]:
#how many breakdowns occur each year in brooklyn
bus.filter(col("Town")=="Brooklyn").groupBy("School_Year").agg(sum("Busbreakdown_ID").alias("Total")).orderBy("School_Year").explain()

== Physical Plan ==
*(3) Sort [School_Year#2411 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(School_Year#2411 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#2813]
   +- *(2) HashAggregate(keys=[School_Year#2411], functions=[sum(cast(Busbreakdown_ID#2412 as bigint))])
      +- Exchange hashpartitioning(School_Year#2411, 200), ENSURE_REQUIREMENTS, [id=#2809]
         +- *(1) HashAggregate(keys=[School_Year#2411], functions=[partial_sum(cast(Busbreakdown_ID#2412 as bigint))])
            +- *(1) Project [School_Year#2411, Busbreakdown_ID#2412]
               +- *(1) Filter (isnotnull(Town#2418) AND (Town#2418 = Brooklyn))
                  +- FileScan csv [School_Year#2411,Busbreakdown_ID#2412,Town#2418] Batched: false, DataFilters: [isnotnull(Town#2418), (Town#2418 = Brooklyn)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/HI/big data/bus.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Town), EqualTo(Town,Brooklyn)], ReadSchema: struct<School_Year:int,Busbre